In [1]:
#imitialization of the pyspark
import findspark
findspark.init()
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
#creating the psark session
spark =SparkSession.builder.appName("datanalysis").getOrCreate()

24/07/18 18:12:40 WARN Utils: Your hostname, hp resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface wlp2s0)
24/07/18 18:12:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/18 18:12:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/18 18:12:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
csvfile ="/home/christine/Downloads/scooter.csv"

In [5]:
#creating a dataframe
df=spark.read\
   .format("csv")\
   .option("inferschema" ,True)\
   .option("header",True)\
   .load(csvfile)

df.show()

+-----+-------+---------+----------+---------------+---------------+--------+--------------------+--------------------+--------+--------------+
|month|trip_id|region_id|vehicle_id|     started_at|       ended_at|DURATION| start_location_name|   end_location_name| user_id|trip_ledger_id|
+-----+-------+---------+----------+---------------+---------------+--------+--------------------+--------------------+--------+--------------+
|  May|1613335|      202|   9424537|5/21/2019 18:33|5/21/2019 18:40| 0:07:03|1901 Roma Ave NE,...|1899 Roma Ave NE,...| 8417864|       1488546|
|  May|1613639|      202|   9424537|5/21/2019 19:07|5/21/2019 19:12| 0:04:57|1 Domenici Center...|1111 Stanford Dr ...| 8417864|       1488838|
|  May|1613708|      202|   9424537|5/21/2019 19:13|5/21/2019 19:15| 0:01:14|1 Domenici Center...|1 Domenici Center...| 8417864|       1488851|
|  May|1613867|      202|   9424537|5/21/2019 19:29|5/21/2019 19:36| 0:06:58|Rotunda at Scienc...|725 University Bl...| 8417864|       1

In [6]:
#creatin a temp view

df.createOrReplaceTempView("u")

In [7]:
df.printSchema()

root
 |-- month: string (nullable = true)
 |-- trip_id: integer (nullable = true)
 |-- region_id: integer (nullable = true)
 |-- vehicle_id: integer (nullable = true)
 |-- started_at: string (nullable = true)
 |-- ended_at: string (nullable = true)
 |-- DURATION: string (nullable = true)
 |-- start_location_name: string (nullable = true)
 |-- end_location_name: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- trip_ledger_id: integer (nullable = true)



In [8]:
df.columns

['month',
 'trip_id',
 'region_id',
 'vehicle_id',
 'started_at',
 'ended_at',
 'DURATION',
 'start_location_name',
 'end_location_name',
 'user_id',
 'trip_ledger_id']

In [9]:
data = spark.sql ("select * from u")


In [11]:
# Group by 'user_id' and 'region_id' and perform multiple aggregations
agg_df = df.groupBy("user_id", "region_id").agg(
    sum("trip_ledger_id").alias("total_trip_ledger_id"),
    sum("DURATION").alias("total_duration"),
    avg("DURATION").alias("avg_duration"),
    count("trip_id").alias("trip_count")
)

# Show the result
agg_df.show()

+--------+---------+--------------------+--------------+------------+----------+
| user_id|region_id|total_trip_ledger_id|total_duration|avg_duration|trip_count|
+--------+---------+--------------------+--------------+------------+----------+
|35609093|      202|             3417024|          null|        null|         2|
|35731526|      202|             3052473|          null|        null|         2|
|36149678|      202|             5433026|          null|        null|         3|
|35617642|      202|             1554369|          null|        null|         1|
|36361723|      202|             9864688|          null|        null|         6|
|36411067|      202|             3127123|          null|        null|         2|
|35635280|      202|            11936851|          null|        null|         7|
|37734630|      202|            18163212|          null|        null|        11|
|37775635|      202|            12450513|          null|        null|         7|
|37800912|      202|        

In [14]:
# Group by 'user_id' and 'region_id' and perform multiple aggregations
agg_df = df.groupBy("user_id").agg(
    sum("region_id").alias("totalregions"),
   
)

# Show the result
agg_df.show()

+--------+------------+
| user_id|totalregions|
+--------+------------+
|35530940|        2626|
|35530825|         404|
|35941253|         808|
|36353218|         202|
|36365637|         808|
|36431716|         808|
|35618796|        1010|
|36674669|         404|
|36698080|         202|
|36703930|         606|
|36437051|        1414|
|37462382|         202|
|29946205|         404|
|37705747|         606|
|37776273|         202|
|37819021|        1818|
|38001650|        1414|
|38046939|        1616|
|38304165|        1818|
|38555775|         202|
+--------+------------+
only showing top 20 rows



In [17]:
# # Order by 'trip_count' in descending order and get the top user
# top_user_df = agg_df.orderBy("trip_id", ascending=False).limit(1)

# # Show the result
# top_user_df.show()